In [4]:
import pandas as pd
import re
import joblib
import pickle
import scipy.sparse
from scipy.sparse import vstack

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.impute import SimpleImputer
from sklearn.utils import resample

from catboost import CatBoostClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
# !pip install catboost
# !pip install joblib

In [3]:
df = pd.read_excel("merged-armenian-books-dataset.xlsx")
df.head(200)

,Title,Author,Price,More Info,Description,Genre,Publisher,Pages,Reader,Avg. Rating,Number of Ratings,Number of reviews
0,(Չ)Վեպ,Հովակիմյան Հովհաննես,NaN,NaN,Հիսուսի կատուն,Ժամանակակից գրականություն,Անտարես,168,NaN,NaN,NaN,NaN
1,«Ամառ առանց լուսաբաց»-ի թղթե թղթե թերթիկ,Agop J. Hacikyan,NaN,NaN,Միջազգային բեսթսելլերի վերաթողարկում՝ ի հիշատա...,Վեպ,NaN,NaN,NaN,4.40,78 ratings,13 reviews
2,«Հինգի ակումբի» ճանապարհորդությունը,Էնիդ Բլայտոն\n(2),NaN,NaN,««Հինգի ակումբի» ճանապարհորդությունը» անգլիացի...,Մանկական գրականություն,NaN,NaN,Կարինե Հովհաննիսյան\n(100),NaN,NaN,NaN
3,«Սրտով մարդը լեռնաշխարհում» և այլ վաղ պատմությ...,Վիլյամ Սարոյան,NaN,NaN,«Սրտով մարդը լեռնաշխարհում» գրքում հավաքված են...,Դասական գրականություն,NaN,NaN,NaN,4.31,218 ratings,11 reviews
4,«Փարիզի ժամերի» կոշտ կազմը,Alex George,NaN,NaN,Մի օր լույսերի քաղաքում Մի գիշեր կորցրած ժաման...,Ժամանակակից գրականություն,NaN,NaN,NaN,3.70,"14,189 ratings","1,991 reviews"
...,...,...,...,...,...,...,...,...,...,...,...,...
195,Ալ տառը,Հոթորն Նաթանիել,NaN,NaN,NaN,Դասական գրականություն,Էդիթ Պրինտ,264,NaN,NaN,NaN,NaN
196,ԱԼ տառը,Նաթանիել Հոթորն,3990 դրամ,Հրատարակչություն-ՀԱՅՐԱՊԵՏ հրատարակչություն-EAN...,NaN,Դասական գրականություն,NaN,NaN,NaN,NaN,NaN,NaN
197,Ալբերտը՝ հրետանու աստված,Հովսեփյան Զ․,NaN,NaN,Ինչու է ժպտում անցորդը,Ժամանակակից գրականություն,Դարակ,311,NaN,NaN,NaN,NaN
198,Ալեն մոլորակը մեր մոլորակում,Վարդանյան Ա.,NaN,NaN,Նամակ Մատնաչափիկին։Գիրք-նամակ,Ժամանակակից գրականություն,Հեղինակային հրատարակչություն,56,NaN,NaN,NaN,NaN


In [ ]:
# Function to extract numeric value
def extract_numeric(price_with_currency):
    if isinstance(price_with_currency, str):
        parts = price_with_currency.split()
        numeric_part = parts[0]
        return numeric_part
    else:
        return None

# Function to extract text
def extract_text(reader):
    if isinstance(reader, str):
        return re.sub(r'\(\d+\)', '', reader).strip()
    else:
        return None

# Applying the function to the 'Price' column
df['Price'] = df['Price'].apply(extract_numeric)

# Converting the 'Price' column to numeric type, coercing errors to NaN
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Converting numeric values to integer type
df['Price'] = df['Price'].astype('Int64')

# Converting the 'Pages' column to numeric type, coercing errors to NaN
df['Pages'] = pd.to_numeric(df['Pages'], errors='coerce')

# Converting numeric values to integer type
df['Pages'] = df['Pages'].astype('Int64')

# Applying the function to the 'Reader' column
df['Reader'] = df['Reader'].apply(extract_text)

df['Number of Ratings'] = df['Number of Ratings'].apply(extract_numeric)

# Removing non-numeric characters (including commas) and converting to integer
df['Number of Ratings'] = df['Number of Ratings'].str.replace(r'\D', '', regex=True).astype('Int64')

df['Number of reviews'] = df['Number of reviews'].apply(extract_numeric)

df['Number of reviews'] = df['Number of reviews'].str.replace(r'\D', '', regex=True).astype('Int64')

df["Author"] = df["Author"].apply(extract_text)

# Removing content in brackets
df['Title'] = df['Title'].str.replace(r'\([^()]*\)', '', regex=True)

df["Description"] = df["Description"].str.replace(r'\([^()]*\)', '', regex=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              6640 non-null   object 
 1   Author             6581 non-null   object 
 2   Price              1257 non-null   Int64  
 3   More Info          332 non-null    object 
 4   Description        5899 non-null   object 
 5   Genre              6640 non-null   object 
 6   Publisher          1629 non-null   object 
 7   Pages              1566 non-null   Int64  
 8   Reader             871 non-null    object 
 9   Avg. Rating        2719 non-null   float64
 10  Number of Ratings  2719 non-null   Int64  
 11  Number of reviews  2719 non-null   Int64  
dtypes: Int64(4), float64(1), object(7)
memory usage: 648.6+ KB


In [ ]:
# Converting the 'More Info' column to strings
df['More Info'] = df['More Info'].astype(str)

# Extracting information for 'Publisher', 'Year of Publishing', 'Language', and 'Age group'
df['Publisher'] = df['More Info'].str.extract(r'ւթյուն-(.*?)-EAN', expand=False)
df['Year of Publishing'] = df['More Info'].str.extract(r'Տարեթիվ-(\d{4})-', expand=False)
df['Language'] = df['More Info'].str.extract(r'Լեզու-(.*?)-', expand=False)
df['Age group'] = df['More Info'].str.extract(r'Տարիք-(.*?)(?:-|$)', expand=False)
df['Pages'] = df['More Info'].str.extract(r'Էջեր-(\d+)', expand=False)

# Filling missing values with NaN
df[['Publisher', 'Year of Publishing', 'Language', 'Age group', 'Pages']] = df[['Publisher', 'Year of Publishing', 'Language', 'Age group', 'Pages']].replace('', pd.NA)

# Droping the 'More Info' column as it's no longer needed
df.drop(columns=['More Info'], inplace=True)

In [ ]:
df.head(30)

,Title,Author,Price,Description,Genre,Publisher,Pages,Reader,Avg. Rating,Number of Ratings,Number of reviews,Year of Publishing,Language,Age group
0,Վեպ,Հովակիմյան Հովհաննես,<NA>,Հիսուսի կատուն,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
1,«Ամառ առանց լուսաբաց»-ի թղթե թղթե թերթիկ,Agop J. Hacikyan,<NA>,Միջազգային բեսթսելլերի վերաթողարկում՝ ի հիշատա...,Վեպ,NaN,NaN,None,4.40,78,13,NaN,NaN,NaN
2,«Հինգի ակումբի» ճանապարհորդությունը,Էնիդ Բլայտոն,<NA>,««Հինգի ակումբի» ճանապարհորդությունը» անգլիացի...,Մանկական գրականություն,NaN,NaN,Կարինե Հովհաննիսյան,NaN,<NA>,<NA>,NaN,NaN,NaN
3,«Սրտով մարդը լեռնաշխարհում» և այլ վաղ պատմությ...,Վիլյամ Սարոյան,<NA>,«Սրտով մարդը լեռնաշխարհում» գրքում հավաքված են...,Դասական գրականություն,NaN,NaN,None,4.31,218,11,NaN,NaN,NaN
4,«Փարիզի ժամերի» կոշտ կազմը,Alex George,<NA>,Մի օր լույսերի քաղաքում Մի գիշեր կորցրած ժաման...,Ժամանակակից գրականություն,NaN,NaN,None,3.70,14189,1991,NaN,NaN,NaN
5,10 հայ ականավոր թագուհիներ,Արտակ Մովսիսյան,<NA>,NaN,Պատմվածք,NaN,NaN,None,4.44,9,3,NaN,NaN,NaN
6,100 Սոցիալական Նորարարություններ Ֆինլանդիայից ...,Ilkka Taipale,<NA>,"Կարելի է մտածել, թե ինչ ընդհանուր բան ունեն խո...",Ոչ գեղարվեստական գրականություն,NaN,NaN,None,3.55,99,14,NaN,NaN,NaN
7,12-ից հետո կհանդիպենք,Պետրոսյան Խ․,<NA>,Խեղված ողնաշարը երկրի,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
8,12-ից հետո կհանդիպենք,Պետրոսյան Խ․,<NA>,12-ից հետո կհանդիպենք,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
9,150 Հոբելյանական ընտրանի,Հովհաննես Թումանյան,6900,«Ընտրանին» պատրաստվել է հայ գրականության դասակ...,Դասական գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN


In [ ]:
# Finding duplicates based on both 'Title' and 'Author'
duplicate_combination = df.duplicated(subset=['Title', 'Author'], keep=False)
# Subseting the DataFrame to show only the duplicates
duplicates_df_combination = df[duplicate_combination]

duplicates_df_combination

,Title,Author,Price,Description,Genre,Publisher,Pages,Reader,Avg. Rating,Number of Ratings,Number of reviews,Year of Publishing,Language,Age group
7,12-ից հետո կհանդիպենք,Պետրոսյան Խ․,<NA>,Խեղված ողնաշարը երկրի,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
8,12-ից հետո կհանդիպենք,Պետրոսյան Խ․,<NA>,12-ից հետո կհանդիպենք,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
16,20000 լյո ջրի տակ,Ժյուլ Վեռն,<NA>,Վերադարձ,Դասական գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
17,20000 լյո ջրի տակ,Ժյուլ Վեռն,<NA>,Սիդհարթա,Դասական գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
166,Ագնես,Պետեր Շտամ,2400,Սովորական թվացող սիրավեպն անկանխատեսելի ընթացք...,Սիրավեպ,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6554,Ջերմ մարմիններ,Isaac Marion,<NA>,Այժմ հիմնական շարժանկարը Summit Entertainment-...,Սիրավեպ,NaN,NaN,None,3.95,45932,6493,NaN,NaN,NaN
6579,Մեռած որպես դռան մեխ,Charlaine Harris,<NA>,Smalltown կոկտեյլ մատուցողուհի Սուկի Սթեքհաուս...,Սիրավեպ,NaN,NaN,None,3.88,581723,13910,NaN,NaN,NaN
6591,Այս սնամեջ ուխտերը,Lexi Ryan,<NA>,New York Times-ի բեսթսելերների հեղինակ Լեքսի Ռ...,Սիրավեպ,NaN,NaN,None,4.20,58309,2681,NaN,NaN,NaN
6602,Լուսին կանչեց,Patricia Briggs,<NA>,Mercedes Thompson մականունով Mercy-ը Volkswage...,Սիրավեպ,NaN,NaN,None,3.80,254592,7324,NaN,NaN,NaN


In [ ]:
# Finding duplicates based only on 'Title'
duplicate_title = df.duplicated(subset=['Title'], keep=False)

# Subseting the DataFrame to show only the duplicates
duplicates_df_title = df[duplicate_title]

duplicates_df_title

,Title,Author,Price,Description,Genre,Publisher,Pages,Reader,Avg. Rating,Number of Ratings,Number of reviews,Year of Publishing,Language,Age group
7,12-ից հետո կհանդիպենք,Պետրոսյան Խ․,<NA>,Խեղված ողնաշարը երկրի,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
8,12-ից հետո կհանդիպենք,Պետրոսյան Խ․,<NA>,12-ից հետո կհանդիպենք,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
16,20000 լյո ջրի տակ,Ժյուլ Վեռն,<NA>,Վերադարձ,Դասական գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
17,20000 լյո ջրի տակ,Ժյուլ Վեռն,<NA>,Սիդհարթա,Դասական գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
28,731 օր քեզ համար,Տա Թևեր,<NA>,Տիեզերական սագա,Ժամանակակից գրականություն,NaN,NaN,None,NaN,<NA>,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6579,Մեռած որպես դռան մեխ,Charlaine Harris,<NA>,Smalltown կոկտեյլ մատուցողուհի Սուկի Սթեքհաուս...,Սիրավեպ,NaN,NaN,None,3.88,581723,13910,NaN,NaN,NaN
6591,Այս սնամեջ ուխտերը,Lexi Ryan,<NA>,New York Times-ի բեսթսելերների հեղինակ Լեքսի Ռ...,Սիրավեպ,NaN,NaN,None,4.20,58309,2681,NaN,NaN,NaN
6602,Լուսին կանչեց,Patricia Briggs,<NA>,Mercedes Thompson մականունով Mercy-ը Volkswage...,Սիրավեպ,NaN,NaN,None,3.80,254592,7324,NaN,NaN,NaN
6629,Ընտրություն,Նիկոլաս Սփարկս,<NA>,1 Նյու Յորք Թայմսի բեսթսելլեր Նիկոլաս Սփարկսը ...,Սիրավեպ,NaN,NaN,None,4.04,49514,1730,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               6640 non-null   object 
 1   Author              6581 non-null   object 
 2   Price               1257 non-null   Int64  
 3   Description         5899 non-null   object 
 4   Genre               6640 non-null   object 
 5   Publisher           315 non-null    object 
 6   Pages               271 non-null    object 
 7   Reader              871 non-null    object 
 8   Avg. Rating         2719 non-null   float64
 9   Number of Ratings   2719 non-null   Int64  
 10  Number of reviews   2719 non-null   Int64  
 11  Year of Publishing  332 non-null    object 
 12  Language            331 non-null    object 
 13  Age group           329 non-null    object 
dtypes: Int64(3), float64(1), object(10)
memory usage: 745.8+ KB


In [ ]:
df['Pages'] = df['Pages'].astype('Int64')
df['Year of Publishing'] = df['Year of Publishing'].astype('Int64')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               6640 non-null   object 
 1   Author              6581 non-null   object 
 2   Price               1257 non-null   Int64  
 3   Description         5899 non-null   object 
 4   Genre               6640 non-null   object 
 5   Publisher           315 non-null    object 
 6   Pages               271 non-null    Int64  
 7   Reader              871 non-null    object 
 8   Avg. Rating         2719 non-null   float64
 9   Number of Ratings   2719 non-null   Int64  
 10  Number of reviews   2719 non-null   Int64  
 11  Year of Publishing  332 non-null    Int64  
 12  Language            331 non-null    object 
 13  Age group           329 non-null    object 
dtypes: Int64(5), float64(1), object(8)
memory usage: 758.8+ KB


In [ ]:
unique_age_groups = df['Age group'].unique()
print(unique_age_groups)

[nan '16+' '12+' '14+' '8+' '6+' '10+' '18+' '17+']


In [ ]:
# Removing the '+' sign from the 'Age group' column and convert it to integer
df['Age group'] = df['Age group'].str.replace('+', '').astype(float).astype('Int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               6640 non-null   object 
 1   Author              6581 non-null   object 
 2   Price               1257 non-null   Int64  
 3   Description         5899 non-null   object 
 4   Genre               6640 non-null   object 
 5   Publisher           315 non-null    object 
 6   Pages               271 non-null    Int64  
 7   Reader              871 non-null    object 
 8   Avg. Rating         2719 non-null   float64
 9   Number of Ratings   2719 non-null   Int64  
 10  Number of reviews   2719 non-null   Int64  
 11  Year of Publishing  332 non-null    Int64  
 12  Language            331 non-null    object 
 13  Age group           329 non-null    Int64  
dtypes: Int64(6), float64(1), object(7)
memory usage: 765.3+ KB


In [ ]:
df.isnull().sum()

Title                    0
Author                  59
Price                 5383
Description            741
Genre                    0
Publisher             6325
Pages                 6369
Reader                5769
Avg. Rating           3921
Number of Ratings     3921
Number of reviews     3921
Year of Publishing    6308
Language              6309
Age group             6311
dtype: int64

In [ ]:
genre_counts = df['Genre'].value_counts()
print(genre_counts)

Genre
Ժամանակակից գրականություն         1790
Դետեկտիվ և թրիլլեր                1260
Դասական գրականություն             1054
Սիրավեպ                            638
Ֆանտաստիկա                         575
Պատմվածք                           414
Վեպ                                264
Պոեզիա                             188
Մանկական գրականություն             148
Ոչ գեղարվեստական գրականություն      85
Հոգևոր գրականություն                69
Գեղարվեստական գրականություն         45
Դրամատուրգիա                        43
Նովել                               40
Թատերգություն                       12
Գիտական ֆանտաստիկա                   8
Արկածային                            7
Name: count, dtype: int64


In [ ]:
len(genre_counts)

17

In [ ]:
df_copy = df.copy()
df_copy

,Title,Author,Price,Description,Genre,Publisher,Pages,Reader,Avg. Rating,Number of Ratings,Number of reviews,Year of Publishing,Language,Age group
0,Վեպ,Հովակիմյան Հովհաննես,<NA>,Հիսուսի կատուն,Ժամանակակից գրականություն,NaN,<NA>,None,NaN,<NA>,<NA>,<NA>,NaN,<NA>
1,«Ամառ առանց լուսաբաց»-ի թղթե թղթե թերթիկ,Agop J. Hacikyan,<NA>,Միջազգային բեսթսելլերի վերաթողարկում՝ ի հիշատա...,Վեպ,NaN,<NA>,None,4.40,78,13,<NA>,NaN,<NA>
2,«Հինգի ակումբի» ճանապարհորդությունը,Էնիդ Բլայտոն,<NA>,««Հինգի ակումբի» ճանապարհորդությունը» անգլիացի...,Մանկական գրականություն,NaN,<NA>,Կարինե Հովհաննիսյան,NaN,<NA>,<NA>,<NA>,NaN,<NA>
3,«Սրտով մարդը լեռնաշխարհում» և այլ վաղ պատմությ...,Վիլյամ Սարոյան,<NA>,«Սրտով մարդը լեռնաշխարհում» գրքում հավաքված են...,Դասական գրականություն,NaN,<NA>,None,4.31,218,11,<NA>,NaN,<NA>
4,«Փարիզի ժամերի» կոշտ կազմը,Alex George,<NA>,Մի օր լույսերի քաղաքում Մի գիշեր կորցրած ժաման...,Ժամանակակից գրականություն,NaN,<NA>,None,3.70,14189,1991,<NA>,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6635,Չեմպիոն,Marie Lu,<NA>,«Նյու Յորք Թայմս»-ի բեսթսելեր Մարի Լուսի բեսթս...,Սիրավեպ,NaN,<NA>,None,3.85,56734,5278,<NA>,NaN,<NA>
6636,Կյանքի գիրքը,Deborah Harkness,<NA>,«Նյու Յորք Թայմս»-ի 1-ին բեսթսելեր սերիալի եզր...,Սիրավեպ,NaN,<NA>,None,4.32,218061,19450,<NA>,NaN,<NA>
6637,Բելադոննա,Adalyn Grace,<NA>,«Նյու Յորք Թայմս»-ի «Բոլոր աստղերն ու ատամները...,Սիրավեպ,NaN,<NA>,None,4.20,191338,13975,<NA>,NaN,<NA>
6638,Անխիղճ երդումներ,Rebecca Ross,<NA>,«Աստվածային մրցակիցներ»-ում սկսված ինտենսիվ ռո...,Սիրավեպ,NaN,<NA>,None,4.02,87715,16560,<NA>,NaN,<NA>


In [ ]:
# Step 1: Preprocessing
# Dropping irrelevant columns
df = df[['Title', 'Description', 'Genre', 'Author']]  # Keeping only relevant columns

# Dropping rows with NaN values in 'Description'
df = df.dropna(subset=['Description', 'Author'])

# Step 2: Tf-idf Encoding
# Combining 'Title' and 'Description' columns into a single text column
df['Text'] = df['Title'] + ' ' + df['Description'] + ' ' + df['Author']

# Applying Tf-idf encoding
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Text'])

# Step 3: Model Training and Evaluation
# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['Genre'], test_size=0.2, random_state=25)

# Training and evaluating RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=25)
rf_classifier.fit(X_train, y_train)
rf_pred = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
print("RandomForestClassifier Accuracy:", rf_accuracy)

# Training and evaluating DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=25)
dt_classifier.fit(X_train, y_train)
dt_pred = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_pred)
print("DecisionTreeClassifier Accuracy:", dt_accuracy)

# Training and evaluating LogisticRegression
lr_classifier = LogisticRegression(max_iter=1000, random_state=25)
lr_classifier.fit(X_train, y_train)
lr_pred = lr_classifier.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_pred)
print("LogisticRegression Accuracy:", lr_accuracy)

# Training and evaluating Feedforward Neural Network (MLPClassifier)
mlp_classifier = MLPClassifier(random_state=25)
mlp_classifier.fit(X_train, y_train)
mlp_pred = mlp_classifier.predict(X_test)
mlp_accuracy = accuracy_score(y_test, mlp_pred)
print("MLPClassifier Accuracy:", mlp_accuracy)

RandomForestClassifier Accuracy: 0.5542271562766866
DecisionTreeClassifier Accuracy: 0.5123825789923142
LogisticRegression Accuracy: 0.6302305721605466
MLPClassifier Accuracy: 0.7386848847139197


In [ ]:
df_new = df_copy.copy()
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Title               6640 non-null   object 
 1   Author              6581 non-null   object 
 2   Price               1257 non-null   Int64  
 3   Description         5899 non-null   object 
 4   Genre               6640 non-null   object 
 5   Publisher           315 non-null    object 
 6   Pages               271 non-null    Int64  
 7   Reader              871 non-null    object 
 8   Avg. Rating         2719 non-null   float64
 9   Number of Ratings   2719 non-null   Int64  
 10  Number of reviews   2719 non-null   Int64  
 11  Year of Publishing  332 non-null    Int64  
 12  Language            331 non-null    object 
 13  Age group           329 non-null    Int64  
dtypes: Int64(6), float64(1), object(7)
memory usage: 765.3+ KB


## Experimenting with different parameters

In [ ]:
# # Converting NaN values to empty strings in text columns
# text_columns = ['Title', 'Description', 'Author', 'Publisher']
# for col in text_columns:
#     df_new[col] = df_new[col].fillna('')

# # Combining all relevant text columns into a single text column
# df_new['Text'] = df_new[text_columns].apply(lambda x: ' '.join(x), axis=1)

# # Applying Tf-idf encoding
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_new['Text'])

# # Spliting the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_new['Genre'], test_size=0.2, random_state=25)

# Calculating class weights with a balance factor
class_counts = df_new['Genre'].value_counts()
total_samples = len(df_new)
balance_factor = 0.5
class_weights = {cls: total_samples / (num_samples * len(class_counts)) ** balance_factor for cls, num_samples in class_counts.items()}

# Defining CatBoostClassifier with GPU acceleration and adjusted class weights
catboost_classifier = CatBoostClassifier(
    iterations=1500,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    random_strength=1,
    border_count=32,
    eval_metric='AUC',
    task_type='GPU',
    class_weights=class_weights,
    verbose=0,
    use_best_model=True
)


# Training the model with validation set
catboost_classifier.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose_eval=False
)

# Saving the trained model
joblib.dump(catboost_classifier, "catboost-model-unbias.pkl")

# Making predictions
catboost_pred = catboost_classifier.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, catboost_pred)
print("CatBoostClassifier Accuracy:", accuracy)

# Evaluating AUC
auc = roc_auc_score(y_test, catboost_classifier.predict_proba(X_test), multi_class='ovo')
print("CatBoostClassifier AUC:", auc)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


CatBoostClassifier Accuracy: 0.6498493975903614
CatBoostClassifier AUC: 0.49079977604671565


In [ ]:
# # Converting NaN values to empty strings in text columns
# text_columns = ['Title', 'Description', 'Author', 'Publisher']
# for col in text_columns:
#     df_new[col] = df_new[col].fillna('')

# # Combining all relevant text columns into a single text column
# df_new['Text'] = df_new[text_columns].apply(lambda x: ' '.join(x), axis=1)

# # Applying Tf-idf encoding
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_new['Text'])

# Saving the TfidfVectorizer
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_new['Genre'], test_size=0.2, random_state=25)

# Defining CatBoostClassifier with GPU acceleration
catboost_classifier = CatBoostClassifier(
    iterations=1600,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    random_strength=1,
    border_count=32,
    eval_metric='AUC',
    task_type='GPU',
    verbose=0
)

# Training the model with validation set
catboost_classifier.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose_eval=False
)

# Saving the trained model
joblib.dump(catboost_classifier, "catboost-model.pkl")

# Making predictions
catboost_pred = catboost_classifier.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, catboost_pred)
print("CatBoostClassifier Accuracy:", accuracy)

# Evaluating AUC
auc = roc_auc_score(y_test, catboost_classifier.predict_proba(X_test), multi_class='ovo')
print("CatBoostClassifier AUC:", auc)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


CatBoostClassifier Accuracy: 0.6069277108433735
CatBoostClassifier AUC: 0.8864871872762049


In [ ]:
# # Converting NaN values to empty strings in text columns
# text_columns = ['Title', 'Description', 'Author', 'Publisher']
# for col in text_columns:
#     df_new[col] = df_new[col].fillna('')

# # Combining all relevant text columns into a single text column
# df_new['Text'] = df_new[text_columns].apply(lambda x: ' '.join(x), axis=1)

# # Applying Tf-idf encoding
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_new['Text'])

# Saving the TfidfVectorizer
with open("tfidf_vectorizer-new.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_new['Genre'], test_size=0.2, random_state=25)

# Defining CatBoostClassifier with GPU acceleration
catboost_classifier = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    random_strength=1,
    border_count=32,
    eval_metric='AUC',
    task_type='GPU',
    verbose=0
)

# Training the model with validation set
catboost_classifier.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose_eval=False
)

# Saving the trained model
joblib.dump(catboost_classifier, "catboost-model-merged.pkl")

# Making predictions
catboost_pred = catboost_classifier.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, catboost_pred)
print("CatBoostClassifier Accuracy:", accuracy)

# Evaluating AUC
auc = roc_auc_score(y_test, catboost_classifier.predict_proba(X_test), multi_class='ovo')
print("CatBoostClassifier AUC:", auc)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


CatBoostClassifier Accuracy: 0.6325301204819277
CatBoostClassifier AUC: 0.8997397884768844


In [ ]:
# Converting NaN values to empty strings in text columns
text_columns = ['Title', 'Description', 'Author', 'Publisher']
for col in text_columns:
    df_new[col] = df_new[col].fillna('')

# Combining all relevant text columns into a single text column
df_new['Text'] = df_new[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Applying Tf-idf encoding
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_new['Text'])

# Saving the TfidfVectorizer
with open("tfidf_vectorizer-final.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_new['Genre'], test_size=0.2, random_state=25)

# Defining CatBoostClassifier with GPU acceleration
catboost_classifier = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    random_strength=1,
    border_count=32,
    eval_metric='AUC',
    task_type='GPU',
    verbose=0
)

# Training the model with validation set
catboost_classifier.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose_eval=False
)

# Saving the trained model
joblib.dump(catboost_classifier, "catboost-model-final.pkl")

# Making predictions
catboost_pred = catboost_classifier.predict(X_test)

# Calculating metrics
accuracy = accuracy_score(y_test, catboost_pred)
print("CatBoostClassifier Accuracy:", accuracy)

# Calculating AUC
auc = roc_auc_score(y_test, catboost_classifier.predict_proba(X_test), multi_class='ovo')
print("CatBoostClassifier AUC:", auc)

# Calculating additional metrics
report = classification_report(y_test, catboost_pred)
print("Classification Report:")
print(report)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


CatBoostClassifier Accuracy: 0.6408132530120482
CatBoostClassifier AUC: 0.887604595075037
Classification Report:
                                precision    recall  f1-score   support

                     Արկածային       0.00      0.00      0.00         2
   Գեղարվեստական գրականություն       0.00      0.00      0.00         6
            Գիտական ֆանտաստիկա       0.00      0.00      0.00         3
         Դասական գրականություն       0.78      0.40      0.53       226
            Դետեկտիվ և թրիլլեր       0.65      0.82      0.73       252
                  Դրամատուրգիա       0.00      0.00      0.00        11
                 Թատերգություն       1.00      0.67      0.80         3
     Ժամանակակից գրականություն       0.59      0.93      0.72       356
          Հոգևոր գրականություն       1.00      0.40      0.57        10
        Մանկական գրականություն       0.94      0.47      0.62        32
                         Նովել       1.00      0.42      0.59        12
Ոչ գեղարվեստական գրակա

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# # Converting NaN values to empty strings in text columns
# text_columns = ['Title', 'Description', 'Author', 'Publisher']
# for col in text_columns:
#     df_new[col] = df_new[col].fillna('')

# # Combining all relevant text columns into a single text column
# df_new['Text'] = df_new[text_columns].apply(lambda x: ' '.join(x), axis=1)

# # Applying Tf-idf encoding
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df_new['Text'])

# Saving the TfidfVectorizer
with open("tfidf_vectorizer-update.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_new['Genre'], test_size=0.2, random_state=25)

# Defining CatBoostClassifier with GPU acceleration
catboost_classifier = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=10,
    l2_leaf_reg=5,
    random_strength=1,
    border_count=40,
    eval_metric='AUC',
    task_type='GPU',
    verbose=0
)

# Training the model with validation set
catboost_classifier.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose_eval=False
)

# Saving the trained model
joblib.dump(catboost_classifier, "catboost-model-merged-new.pkl")

# Making predictions
catboost_pred = catboost_classifier.predict(X_test)

# Evaluating accuracy
accuracy = accuracy_score(y_test, catboost_pred)
print("CatBoostClassifier Accuracy:", accuracy)

# Evaluating AUC
auc = roc_auc_score(y_test, catboost_classifier.predict_proba(X_test), multi_class='ovo')
print("CatBoostClassifier AUC:", auc)

Default metric period is 5 because AUC is/are not implemented for GPU
AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


CatBoostClassifier Accuracy: 0.6340361445783133
CatBoostClassifier AUC: 0.8783532553521759


### Evaluating Accuracy and Multiclass AUC with other models

In [ ]:
# Step 1: Preprocessing
# Keeping top 15 genres and filter out others
top_genres = df_copy['Genre'].value_counts().nlargest(15).index
df_copy = df_copy[df_copy['Genre'].isin(top_genres)]

# Droping irrelevant columns
df_copy = df_copy[['Title', 'Description', 'Genre', 'Author']]

# Droping rows with NaN values in 'Description'
df_copy = df_copy.dropna(subset=['Description', 'Author'])

# Step 2: Tf-idf Encoding
# Combining 'Title' and 'Description' columns into a single text column
df_copy['Text'] = df_copy['Title'] + ' ' + df_copy['Description'] + ' ' + df_copy['Author']

# Applying Tf-idf encoding
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_copy['Text'])

# Step 3: Model Training and Evaluation
# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_copy['Genre'], test_size=0.2, random_state=25)

# Handling class imbalance by upsampling minority classes
X_train_upsampled = []
y_train_upsampled = []
max_samples = X_train[y_train == top_genres[0]].shape[0]
for genre in top_genres:
    X_genre = X_train[y_train == genre]
    y_genre = y_train[y_train == genre]
    X_genre_upsampled, y_genre_upsampled = resample(X_genre, y_genre, replace=True, n_samples=max_samples, random_state=25)
    X_train_upsampled.append(X_genre_upsampled)
    y_train_upsampled.append(y_genre_upsampled)

X_train_upsampled = vstack(X_train_upsampled)
y_train_upsampled = np.hstack(y_train_upsampled)

# Training and evaluating RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=25)
rf_classifier.fit(X_train_upsampled, y_train_upsampled)
rf_train_pred = rf_classifier.predict(X_train)
rf_train_probs = rf_classifier.predict_proba(X_train)
rf_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), rf_train_probs, average='macro')
print("RandomForestClassifier Train Accuracy:", accuracy_score(y_train, rf_train_pred))
print("RandomForestClassifier Train Multiclass AUC:", rf_train_auc_score)

rf_pred = rf_classifier.predict(X_test)
rf_probs = rf_classifier.predict_proba(X_test)
rf_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), rf_probs, average='macro')
print("RandomForestClassifier Test Accuracy:", accuracy_score(y_test, rf_pred))
print("RandomForestClassifier Test Multiclass AUC:", rf_auc_score)

# Training and evaluating DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=25)
dt_classifier.fit(X_train_upsampled, y_train_upsampled)
dt_train_pred = dt_classifier.predict(X_train)
dt_train_probs = dt_classifier.predict_proba(X_train)
dt_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), dt_train_probs, average='macro')
print("DecisionTreeClassifier Train Accuracy:", accuracy_score(y_train, dt_train_pred))
print("DecisionTreeClassifier Train Multiclass AUC:", dt_train_auc_score)

dt_pred = dt_classifier.predict(X_test)
dt_probs = dt_classifier.predict_proba(X_test)
dt_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), dt_probs, average='macro')
print("DecisionTreeClassifier Test Accuracy:", accuracy_score(y_test, dt_pred))
print("DecisionTreeClassifier Test Multiclass AUC:", dt_auc_score)

# Training and evaluating LogisticRegression
lr_classifier = LogisticRegression(max_iter=1000, random_state=25)
lr_classifier.fit(X_train_upsampled, y_train_upsampled)
lr_train_pred = lr_classifier.predict(X_train)
lr_train_probs = lr_classifier.predict_proba(X_train)
lr_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), lr_train_probs, average='macro')
print("LogisticRegression Train Accuracy:", accuracy_score(y_train, lr_train_pred))
print("LogisticRegression Train Multiclass AUC:", lr_train_auc_score)

lr_pred = lr_classifier.predict(X_test)
lr_probs = lr_classifier.predict_proba(X_test)
lr_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), lr_probs, average='macro')
print("LogisticRegression Test Accuracy:", accuracy_score(y_test, lr_pred))
print("LogisticRegression Test Multiclass AUC:", lr_auc_score)

# Training and evaluating Feedforward Neural Network (MLPClassifier)
mlp_classifier = MLPClassifier(random_state=25)
mlp_classifier.fit(X_train_upsampled, y_train_upsampled)
mlp_train_pred = mlp_classifier.predict(X_train)
mlp_train_probs = mlp_classifier.predict_proba(X_train)
mlp_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), mlp_train_probs, average='macro')
print("MLPClassifier Train Accuracy:", accuracy_score(y_train, mlp_train_pred))
print("MLPClassifier Train Multiclass AUC:", mlp_train_auc_score)

mlp_pred = mlp_classifier.predict(X_test)
mlp_probs = mlp_classifier.predict_proba(X_test)
mlp_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), mlp_probs, average='macro')
print("MLPClassifier Test Accuracy:", accuracy_score(y_test, mlp_pred))
print("MLPClassifier Test Multiclass AUC:", mlp_auc_score)

RandomForestClassifier Train Accuracy: 0.934068406840684
RandomForestClassifier Train Multiclass AUC: 0.5064453996191987
RandomForestClassifier Test Accuracy: 0.6660666066606661
RandomForestClassifier Test Multiclass AUC: 0.5413807664141381
DecisionTreeClassifier Train Accuracy: 0.8906390639063907
DecisionTreeClassifier Train Multiclass AUC: 0.5516343460844817
DecisionTreeClassifier Test Accuracy: 0.5013501350135013
DecisionTreeClassifier Test Multiclass AUC: 0.5190828239792109
LogisticRegression Train Accuracy: 0.9270927092709271
LogisticRegression Train Multiclass AUC: 0.5086377895448544
LogisticRegression Test Accuracy: 0.7542754275427542
LogisticRegression Test Multiclass AUC: 0.5452573396820564


In [ ]:
# Step 1: Preprocessing
# Keeping top 15 genres and filter out others
top_genres = df_copy['Genre'].value_counts().nlargest(10).index
df_copy = df_copy[df_copy['Genre'].isin(top_genres)]

# Droping irrelevant columns
df_copy = df_copy[['Title', 'Description', 'Genre', 'Author']]

# Droping rows with NaN values in 'Description'
df_copy = df_copy.dropna(subset=['Description', 'Author'])

# Step 2: Tf-idf Encoding
# Combining 'Title' and 'Description' columns into a single text column
df_copy['Text'] = df_copy['Title'] + ' ' + df_copy['Description'] + ' ' + df_copy['Author']

# Applying Tf-idf encoding
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_copy['Text'])

# Step 3: Model Training and Evaluation
# Spliting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df_copy['Genre'], test_size=0.2, random_state=25)

# Handling class imbalance by upsampling minority classes
X_train_upsampled = []
y_train_upsampled = []
max_samples = X_train[y_train == top_genres[0]].shape[0]
for genre in top_genres:
    X_genre = X_train[y_train == genre]
    y_genre = y_train[y_train == genre]
    X_genre_upsampled, y_genre_upsampled = resample(X_genre, y_genre, replace=True, n_samples=max_samples, random_state=25)
    X_train_upsampled.append(X_genre_upsampled)
    y_train_upsampled.append(y_genre_upsampled)

X_train_upsampled = vstack(X_train_upsampled)
y_train_upsampled = np.hstack(y_train_upsampled)

# Training and evaluating RandomForestClassifier with adjusted max_depth and min_samples_split
rf_classifier = RandomForestClassifier(random_state=25, max_depth=20, min_samples_split=5)
rf_classifier.fit(X_train_upsampled, y_train_upsampled)
rf_train_pred = rf_classifier.predict(X_train)
rf_train_probs = rf_classifier.predict_proba(X_train)
rf_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), rf_train_probs, average='macro')
print("RandomForestClassifier Train Accuracy:", accuracy_score(y_train, rf_train_pred))
print("RandomForestClassifier Train Multiclass AUC:", rf_train_auc_score)

rf_pred = rf_classifier.predict(X_test)
rf_probs = rf_classifier.predict_proba(X_test)
rf_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), rf_probs, average='macro')
print("RandomForestClassifier Test Accuracy:", accuracy_score(y_test, rf_pred))
print("RandomForestClassifier Test Multiclass AUC:", rf_auc_score)


# Training and evaluating DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=25, max_depth=60, min_samples_split=20)
dt_classifier.fit(X_train_upsampled, y_train_upsampled)
dt_train_pred = dt_classifier.predict(X_train)
dt_train_probs = dt_classifier.predict_proba(X_train)
dt_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), dt_train_probs, average='macro')
print("DecisionTreeClassifier Train Accuracy:", accuracy_score(y_train, dt_train_pred))
print("DecisionTreeClassifier Train Multiclass AUC:", dt_train_auc_score)

dt_pred = dt_classifier.predict(X_test)
dt_probs = dt_classifier.predict_proba(X_test)
dt_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), dt_probs, average='macro')
print("DecisionTreeClassifier Test Accuracy:", accuracy_score(y_test, dt_pred))
print("DecisionTreeClassifier Test Multiclass AUC:", dt_auc_score)

# Training and evaluating LogisticRegression
lr_classifier = LogisticRegression(max_iter=1000, random_state=25, C=0.01)  # Adjust C for regularization
lr_classifier.fit(X_train_upsampled, y_train_upsampled)
lr_train_pred = lr_classifier.predict(X_train)
lr_train_probs = lr_classifier.predict_proba(X_train)
lr_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), lr_train_probs, average='macro')
print("LogisticRegression Train Accuracy:", accuracy_score(y_train, lr_train_pred))
print("LogisticRegression Train Multiclass AUC:", lr_train_auc_score)

lr_pred = lr_classifier.predict(X_test)
lr_probs = lr_classifier.predict_proba(X_test)
lr_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), lr_probs, average='macro')
print("LogisticRegression Test Accuracy:", accuracy_score(y_test, lr_pred))
print("LogisticRegression Test Multiclass AUC:", lr_auc_score)

# Training and evaluating Feedforward Neural Network (MLPClassifier) with L2 regularization and dropout
mlp_classifier = MLPClassifier(random_state=25, alpha=0.0001, hidden_layer_sizes=(512,))
mlp_classifier.fit(X_train_upsampled, y_train_upsampled)

mlp_train_pred = mlp_classifier.predict(X_train)
mlp_train_probs = mlp_classifier.predict_proba(X_train)
mlp_train_auc_score = roc_auc_score(label_binarize(y_train, classes=top_genres), mlp_train_probs, average='macro')
print("MLPClassifier Train Accuracy:", accuracy_score(y_train, mlp_train_pred))
print("MLPClassifier Train Multiclass AUC:", mlp_train_auc_score)

mlp_pred = mlp_classifier.predict(X_test)
mlp_probs = mlp_classifier.predict_proba(X_test)
mlp_auc_score = roc_auc_score(label_binarize(y_test, classes=top_genres), mlp_probs, average='macro')
print("MLPClassifier Test Accuracy:", accuracy_score(y_test, mlp_pred))
print("MLPClassifier Test Multiclass AUC:", mlp_auc_score)

RandomForestClassifier Train Accuracy: 0.7558695652173913
RandomForestClassifier Train Multiclass AUC: 0.46306767763256457
RandomForestClassifier Test Accuracy: 0.4826086956521739
RandomForestClassifier Test Multiclass AUC: 0.5194863127034676
DecisionTreeClassifier Train Accuracy: 0.6545652173913044
DecisionTreeClassifier Train Multiclass AUC: 0.44512777134417764
DecisionTreeClassifier Test Accuracy: 0.37217391304347824
DecisionTreeClassifier Test Multiclass AUC: 0.5064663487793492
LogisticRegression Train Accuracy: 0.7208695652173913
LogisticRegression Train Multiclass AUC: 0.5138537098448075
LogisticRegression Test Accuracy: 0.5765217391304348
LogisticRegression Test Multiclass AUC: 0.535513429331419
MLPClassifier Train Accuracy: 0.9408695652173913
MLPClassifier Train Multiclass AUC: 0.4033310506718889
MLPClassifier Test Accuracy: 0.7252173913043478
MLPClassifier Test Multiclass AUC: 0.4964803790694109
